In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import glob
import ipywidgets as widgets

file_uploader = widgets.FileUpload(multiple = True)

def on_file_upload(change):
    global select_file, current_file
    
    for details_dict in change['new']:
        if details_dict['name'][-3:] == 'asc'or details_dict['name'][-3:] == 'txt':
            NewFileName = './AKTAfiles/' + details_dict['name']
            files.append(NewFileName)
        with open(NewFileName, "wb") as f:
            f.write(details_dict['content'])

    current_file = select_file.value
    select_file.options=tuple(files)


def on_select_file(change):
    global current_file
    current_file = change['new']

from pathlib import Path
Path('./AKTAfiles').mkdir(parents=True, exist_ok = True)

files = glob.glob("./AKTAfiles/*.asc") + glob.glob("./AKTAfiles/*.txt")

current_file = None

select_markdown = widgets.HTML(value="Use the <b>Upload</b> button to "
                               "upload your files if they are not among the choices you see "
                               "listed below.<br/>Click on the file name in the list below "
                               "to select the <b>AKTA run</b> to plot. "
                               " <br>Run the the next cell to proceed. <br> <br>"
                               " You can change selection without running this cell again. <br> <br>")


select_file = widgets.Select(options = files, description = 'AKTA runs:', layout={'width':'600px'})

hbox = widgets.HBox([file_uploader, select_file])
vbox = widgets.VBox([select_markdown, hbox])

display(vbox)

file_uploader.observe(on_file_upload, 'value')
select_file.observe(on_select_file, 'value')

def plot_elution(
          filename,
          B_elution=False,
          graphTitle='output',
          savefile=False,
          savescale=2,
          bgcolor='AliceBlue'
                                ):

    df = pd.read_table(filename, header=None, skiprows=1)#, usecols=range(ncols))
    df = df[list(range(0,df.count(axis='columns').max()))]
    index1 = list(filter(lambda x: x ==x, df.iloc[0].tolist()))
    new_index1 = []
    for item in index1:
        new_index1.append(item)
        new_index1.append(item)
    index2 = list(filter(lambda x: x ==x, df.iloc[1].tolist()))
    
    arrays = [new_index1,index2]
    multi_index =pd.MultiIndex.from_arrays(arrays,names=['channel','xy'])
    
    df = df.iloc[2:]
    df.columns = multi_index
    df = df.astype(float)
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for plot in index1:
        if plot[:2] == 'UV':
            fig.add_trace(
                go.Scatter(x=df[plot,"ml"], y=df[plot,"mAU"], name= plot[:3] + plot[-3:]),
                                secondary_y=False)
        elif plot[:4] == 'Conc':
            # Use add_trace function and specify secondary_y axes = True.
            B_elution == True
            fig.add_trace(
                go.Scatter(x=df[plot,"ml"], y=df[plot,"%"], name=plot),
                secondary_y=True,)
            
    fig.update_layout(
                        title_text=graphTitle,
                        template = 'simple_white',
                        height = 400,
                        width = 700,
                        xaxis = dict(title_text="volume (mL)",
                                     gridcolor='light gray',
                                     gridwidth = 0.3,
                                     showgrid = True,
                                     title_font_size = 16, 
                                     mirror = True),
                        yaxis = dict(title_text = "Abs (mAU)",
                                     gridcolor='light gray',
                                     gridwidth = 0.3,
                                     showgrid = True,
                                     title_font_size = 16, 
                                     mirror = True),
                        paper_bgcolor=bgcolor
                     )
    if B_elution:
        fig.update_yaxes(title_text="Conc B (%)", secondary_y=True)

    if savefile:  
            import plotly.io as pio
            pio.write_image(fig, graphTitle+'.png', scale = savescale)
    fig.show()

In [ ]:
# import graph_objects from plotly package
filename = select_file.value

title = filename[:-4]
plot_elution(filename,
             graphTitle=title,
            #  B_elution=select_Belution.value,
             savefile=False,
             #bgcolor='white'
             )